Oppgave 1.7: Hvor mange elbiler ble førstegangsregistrert i 2022
    Leser fra preppet parquet fil laget i oppg. 1.6

In [1]:
import polars as pl

kp = pl.read_parquet('kjoretoy_preppet.parquet')

df_ant_el_2022 = kp.filter(
    (pl.col('tekn_reg_f_g_n').cast(str).str.contains('2022')) 
    & (pl.col('Er_Elbil') == True)
)

print(f"Antall Elbiler registrert i 2022 er: {df_ant_el_2022.height}")

Antall Elbiler registrert i 2022 er: 37249


Oppgave 1.8: Hvor mange prosent av persbilene solgt i 2022 var elbiler
    Her leser vi fra tekn_reg_eier_dato og sjekker at datoen starter med året 2022, da det er slik dataen er strukturert.

In [42]:
antallElbil = len(kp.filter((pl.col('tekn_reg_eier_dato').cast(str).str.starts_with('2022')) & (pl.col('Er_Elbil') == True)))

antallBilerTotalt = len(kp.filter(pl.col('tekn_reg_eier_dato').cast(str).str.starts_with('2022')))

prosent =((antallElbil / antallBilerTotalt) * 100).__round__(2)

print(f"Antall Elbiler registrert på ny eier i 2022: {antallElbil}")
print(f"Antall alle biler registrert på ny eier i 2022: {antallBilerTotalt}")
print(f"Prosent som er elbil: {prosent}")

Antall Elbiler registrert på ny eier i 2022: 38670
Antall alle biler registrert på ny eier i 2022: 133994
Prosent som er elbil: 28.86


Oppg 1.9: Hvilken bilmodell var mest populær i 2022
    Jeg tolker denne oppgaven til å bety mest populær å kjøpe, ergo vi ser på førstegangsregistrerte biler. 
    Vi filtrerer ut alle biler fra 2022, og teller antall hendelser av modellen. Deretter sorterer vi listen etter count, synkende.
    Hjelpedatasett Kjøretøygrupper er brukt ved siden av. 

In [46]:
biler2022 = kp.filter(pl.col('tekn_reg_f_g_n').cast(str).str.starts_with('2022') & (pl.col('tekn_kjtgrp') == 101))

populæreBiler = biler2022.group_by("tekn_modell").count()

populæreBiler.sort(pl.col('count'), descending=True)

tekn_modell,count
str,u32
"""Model Y""",8719
"""ID.4 GTX 220 K…",4021
"""XC40""",3001
"""IONIQ5""",2890
"""ENYAQ 80X""",2799
"""Polestar 2""",2413
"""Mustang Mach-E…",2389
"""RAV4""",2009
"""E TRON 55""",1975


Oppg 1.10: Hvor mange gule kjøretøy ble solgt i mai 2022?
    Sorterer datasettet på førstegangsregistrerte biler i mai, 2022, som også er av typen personbil. 
    For gøy regner jeg ut både antall gule personbiler, og gule kjøretøy av alle typer totalt. 
    Hjelpedatasett Fargekoder er brukt ved siden av. 

In [71]:
personbiler = kp.filter(pl.col('tekn_reg_f_g_n').cast(str).str.starts_with('202205') & (pl.col('tekn_kjtgrp') == 101))
allekjøretøy = kp.filter(pl.col('tekn_reg_f_g_n').cast(str).str.starts_with('202205'))

gulebiler = len(personbiler.filter(pl.col('tekn_farge').cast(str).str.contains('09')))
gulekjøretøy = len(allekjøretøy.filter(pl.col('tekn_farge').cast(str).str.contains('09')))

print(f"Antall gule personbiler: {gulebiler}")
print(f"Antall gule kjøretøy totalt: {gulekjøretøy}")

Antall gule personbiler: 33
Antall gule kjøretøy totalt: 74


Oppg 1.11: Hvor stor andel av personbilene som selges i Norge har firehjulstrekk?
    Siden det ikke defineres noe årstall for salg, så tolker jeg at det betyr alle personbilene noen gang registrert i Norge.
    Vi filtrerer først ut alle personbiler, og deretter ut alle som ikke har 2 aksler i drift. 

In [80]:
personbilerAlle = kp.filter(pl.col('tekn_kjtgrp') == 101)

firehjulstrekk = len(personbilerAlle.filter(pl.col('tekn_aksler_drift') == 2))

print(f"Antall biler solgt totalt i Norge med firehjulstrekk: {firehjulstrekk}")

Antall biler solgt totalt i Norge med firehjulstrekk: 192688


Oppg 1.12: Hvilken måned i året førstegangsregistreres det flest biler i Norge?
    I denne oppgaven valgte vi å returnere en tabell over alle 12 måneder som viser antall registrerte biler totalt i datasett i denne måneden. 
    Siden det ikke defineres noe spesielt kjøretøy, valgte vi å definere "bil" som alt i hjelpedatasettet Kjøretøygrupper som inneholder ordet "bil". (koder: 101-381)
    Vi omformer dato til Date, og henter ut måneden. Vi filtrerer ut alle kjøretøy som har gruppe lik eller mindre enn 381. Deretter grupperer vi på måneden, og teller antall som inneholder den måneden. Deretter sorterer vi på count, slik at de mest populære månedene kommer først. 
    Dette gjelder for alle år sammenlagt.
    

In [117]:
dato = kp.with_columns(
    måned = pl.col('tekn_reg_f_g_n').cast(str).str.strptime(pl.Date, format='%Y %m %d', strict=False).dt.month(), 
)

verifiserteBiler = dato.filter(pl.col('tekn_kjtgrp') <= 381)

verifiserteBiler.group_by('måned').count().sort('count', descending=True)

måned,count
i8,u32
12,57978
6,41440
3,40727
9,38833
11,37217
8,35557
5,31839
10,31400
4,27959


Oppg 1.13: Hva var den mest populære fargen for førstegangsregistrerte biler i Norge i hvert år?
    Denne oppgaven gjorde vi omtrent samtidig med 1.16, og logikken er derfor den samme. Referte til hjelpedatasett Fargekoder: 06 - Svart, 05 - Grå.
    

In [47]:
alleBiler = kp.with_columns(
    år = pl.col('tekn_reg_f_g_n').cast(str).str.strptime(pl.Date, format='%Y %m %d', strict=False).dt.year(),
)
grupperteBiler = alleBiler.group_by('år').count().sort('count', descending=True)
                                                      
topp5Farger = []
alleÅr = grupperteBiler.head(5)

for år in alleÅr['år']:
    filtrertDf = alleBiler.filter(pl.col('år') == år)
    grupperteFarger = filtrertDf.group_by('tekn_farge').count().sort('count', descending=True)

    mestPopFarge = grupperteFarger.head(1)['tekn_farge'][0]
    topp5Farger.append(mestPopFarge)

mestPopBilMedMestPopFarge = pl.DataFrame({'år': alleÅr['år'], 'mest_populære_farge': topp5Farger})

mestPopBilMedMestPopFarge.sort('år', descending=True)


år,mest_populære_farge
i32,str
2024,"""06"""
2023,"""06"""
2022,"""06"""
2021,"""05"""
2020,"""05"""


Oppg 1.14: Hvilken farge var den mest populære for traktorer i 2022?
    Hjelpedatasettet Kjøretøygrupper er brukt.
    Jeg tolker som at det gjelder for traktorer med ny eier i 2022. Først filtrer jeg ut alle traktorer, så filtrerer jeg alle traktorer med nyregistrert eier i 2022. For å vise frem riktig tall pr farge, grupperer jeg på farge og sorterer slik at høyeste tallet kommer først. Siden jeg ikke har med årstallet i tabellen, gir jeg tekn_farge et passende forklarende alias.

In [14]:
alleTraktorer = kp.filter((pl.col('tekn_kjtgrp') == 401))
traktorerI22 = alleTraktorer.filter(pl.col('tekn_reg_eier_dato').cast(str).str.strptime(pl.Date, format='%Y %m %d').dt.year() == 2022)

mestPopFarge = traktorerI22.group_by(pl.col('tekn_farge').alias('Mest_Pop_Farge_2022')).count().sort('count', descending=True)

mestPopFarge.limit(1)


Mest_Pop_Farge_2022,count
str,u32
"""04""",1740


Oppg 1.15: Hvilke bilmerker var de 5 mest populære i 2022?
    Siden ikke "bil" er videre definert, gjør jeg som i tidligere oppgave, og definerer bil som alle kjøretøygrupper bestående av kjøretøy som inneholder ordet "bil". (Gruppe 101 - 381)
    Siden 1.16 bygger på denne oppg, så bruker jeg førstegangsregistrert i Norge 2022 som utgangspunkt for dato.
    Jeg filterer først ut alle biler, så ut de med riktig årstall. Deretter grupperer jeg på merke, og teller, og deretter begrenser jeg resultat-tabellen til 5 rader. 
    

In [16]:
bilmerker = kp.filter(pl.col('tekn_kjtgrp') <= 381)

bilmerkerI22 = bilmerker.filter(pl.col('tekn_reg_f_g_n').cast(str).str.strptime(pl.Date, format='%Y %m %d').dt.year() == 2022)

resultat = bilmerker.group_by(pl.col('tekn_merke').alias('merke')).count().sort('count', descending=True)

resultat.limit(5)

merke,count
i64,u32
5750,51988
5480,40368
4360,38966
3900,28341
5800,28069


Oppg 1.16: Hva var den mest populære fargen for de fem mest populære bilmerkene i 2022 (førstegangsregistrert i 2022)?
    Denne oppg bygger videre på dataframe laget i forrige oppgave. Først så grupperer vi etter merke, og teller merkene slik som tidligere. Vi legger de 5 mest populære inn i en variabel, og itererer over den. For hver bil i Topp5Biler, filtrer original dataframe slik at den består bare av bilmerket. Deretter, gjør en rangering av fargene, og plukk ut den mest populære - så legg den i en liste over topp5Farger.
    Til slutt, lag en NY dataframe med innholdet. For gøy, sorterte jeg resultattabellen for å vise bilmerkekode synkende. 

In [46]:
grupperteBiler = bilmerkerI22.group_by('tekn_merke').count().sort('count', descending=True)
                                                      
topp5Farger = []
topp5Biler = grupperteBiler.head(5)

for bil in topp5Biler['tekn_merke']:
    filtrerteBiler = bilmerkerI22.filter(pl.col('tekn_merke') == bil)
    grupperteFarger = filtrerteBiler.group_by('tekn_farge').count().sort('count', descending=True)

    mestPopFarge = grupperteFarger.head(1)['tekn_farge'][0]
    topp5Farger.append(mestPopFarge)

mestPopBilMedMestPopFarge = pl.DataFrame({'tekn_merke': topp5Biler['tekn_merke'], 'mest_populære_farge': topp5Farger})

mestPopBilMedMestPopFarge.sort('tekn_merke', descending=True)


tekn_merke,mest_populære_farge
i64,str
5800,"""06"""
5750,"""01"""
5480,"""05"""
4360,"""06"""
3900,"""05"""
